The same libraries were used when webscrapping was performed for hotels

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In this case it took a larger amount of headers to access the site information.

In [6]:
headers = {
    "Access-Control-Allow-Origin": "*",
    "Access-Control-Allow-Methods": "GET",
    "Access-Control-Allow-Headers": "Content-Type",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate",
    "accept-language": "en-GB,en;q=0.9,en-US;q=0.8",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43",
}
url = "https://www.booking.com/searchresults.pt-pt.html?aid=375654&label=msn-jrwrFdUb9zKNuCHIkGmz2g-80745410834424%3Atikwd-80745436851624%3Aloc-152%3Aneo%3Amte%3Alp136814%3Adec%3Aqsbooking+&lang=pt-pt&sid=46fbd71f0246b3a03ace98d83fca46ba&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.pt-pt.html%3Faid%3D375654%3Blabel%3Dmsn-jrwrFdUb9zKNuCHIkGmz2g-80745410834424%253Atikwd-80745436851624%253Aloc-152%253Aneo%253Amte%253Alp136814%253Adec%253Aqsbooking%2520%3Bsid%3D46fbd71f0246b3a03ace98d83fca46ba%3Bsb_price_type%3Dtotal%3Bsrpvid%3Dc84b81c8845800cf%26%3B&ss=Beja&is_ski_area=0&ssne=Beja&ssne_untouched=Beja&dest_id=-2159655&dest_type=city&checkin_year=2022&checkin_month=1&checkin_monthday=25&checkout_year=2022&checkout_month=1&checkout_monthday=28&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'lxml')

To access the respective comments of each hotel it was necessary to build a new link to be performed the webscrapping, for this was used a common part for all links and added the name of the hotel, to access your comments specifically

In [7]:
reviews_links = []
for link in soup.findAll('a', {'class': 'fb01724e5b'}):

    a = link['href']
    hotel = a.split('/')[5].split('?')[0]
    a = 'https://www.booking.com/reviews/pt/hotel/' + hotel
    reviews_links.append(a)

In this last part, the comments relating to each website were extracted and the .csv of each of them was created.

In [8]:
count = 0
allreviews = []

for link in reviews_links:
    try:
        response2 = requests.get(link, headers=headers)
        soup2 = BeautifulSoup(response2.content, 'lxml')
        for r in soup2.findAll('span', {'itemprop': 'reviewBody'}):
            try:
                rev = r.text
                allreviews.append(rev + '\n')
            except:
                pass
    except:
        pass
    count += 1
    if allreviews != []:
        seen = set()
        allreviews = [item for item in allreviews if not(
            tuple(item) in seen or seen.add(tuple(item)))]
        dfr = pd.DataFrame.from_dict({'Avaliações': allreviews})
        print(dfr)
        dfr.to_csv('hotel' + str(count) + '.csv')
        allreviews = []


                                           Avaliações
0                de tudo , o edifício é lindíssimo \n
1   Gosto que as camas tenham lençol, além do edre...
2   Travesseiros um bom o outro pela hora da morte...
3   Amei a suntuosidade do local.\nQuarto silencio...
4                                    gostei de tudo\n
5                   localização e espaço da Pousada\n
6                                Temperatura quarto\n
7                                   Globalmente bem\n
8          Acessibilidade para a recepção e quartos\n
9   Localização da pousada, vista do quarto para o...
10  O quarto era arrumado demasiado tarde, por vol...
11  O quarto era extremamente acolhedor e limpo. A...
12  A rede wi-fi é fraca, a vista do quarto , a il...
13  Da amabilidade e simpatia de todo o pessoal es...
14                                do estacionamento\n
15                                 De todo o espaço\n
16  A piscina devia ser limpa com maior regularida...
17  Da simpatia e amabilidad